In [ ]:
"""superposition.ipynb"""
# Cell 1 - Check qiskit install

from __future__ import annotations

import typing

import numpy as np
import qiskit  # type: ignore
from IPython.core.display import Math
from qiskit import Aer, QuantumCircuit, execute  # type: ignore
from qiskit.visualization import plot_bloch_multivector  # type: ignore
from qiskit.visualization import plot_distribution  # type: ignore

if typing.TYPE_CHECKING:
    from typing import Any

    from numpy.typing import NDArray


def display_array(
    a: NDArray[np.complex_], places: int = 5, column: bool = False, prefix: str = ""
) -> None:
    def strip(val: float) -> str:
        frmt: str = ":." + str(places) + "f"
        d: str = str("{v" + frmt + "}").format(v=val)
        while d[-1] == "0":
            d = d[:-1]
        if d[-1] == ".":
            d = d[:-1]
        if float(d) == 0:
            d = "0"
        return d

    m: NDArray[np.complex_] = np.copy(a)
    if len(m.shape) == 1:
        m = m[np.newaxis, :]
        if column:
            m = m.T
    prec: float = 1 / 10**places
    s: str = r"\begin{bmatrix}"
    for row in range(m.shape[0]):
        for col in range(m.shape[1]):
            v: np.complex_ = m[row, col]
            real_comp: float = float(np.real(v))
            imag_comp: float = float(np.imag(v))
            is_imag_neg: bool = imag_comp < 0
            is_real_zero: bool = bool(np.isclose(real_comp, 0, atol=prec))
            is_imag_zero: bool = bool(np.isclose(imag_comp, 0, atol=prec))
            is_imag_one: bool = bool(np.isclose(abs(imag_comp), 1, atol=prec))
            if is_real_zero:
                if is_imag_zero:
                    s += "0"
            else:
                s += strip(real_comp)
            if not is_imag_zero:
                if is_imag_one:
                    if is_imag_neg:
                        s += r"-i"
                    else:
                        if not is_real_zero:
                            s += "+"
                        s += r"i"
                else:
                    if not is_real_zero and not is_imag_neg:
                        s += " + "
                    s += strip(imag_comp) + "i"
            if col < m.shape[1] - 1:
                s += " &"
        s += r"\\"
    s += r"\end{bmatrix}"
    display(Math(prefix + s))


modules: list[tuple[str, int]] = [*qiskit.__qiskit_version__.items()]
for m in modules:
    print(f"Module: {m[0]} : Version: {m[1]}")

In [ ]:
# Cell 2 - One qubit circuit on basis (no measurement)

# Select a quantum computer to use
backend: Any = Aer.get_backend("statevector_simulator")

# Create a new circuit with one qubit
qc: Any = QuantumCircuit(1)

# Initialize the (only) qubit (#0) to a 1 state
qc.initialize([0, 1], 0)
qc.save_statevector("sv1")

# Apply a NOT gate (Pauli X gate)
qc.x(0)
qc.save_statevector("sv2")

# Now run the circuit on the device
result: Any = execute(qc, backend).result()

# Get the state vectors
sv1: Any = result.data(0)["sv1"]
sv2: Any = result.data(0)["sv2"]

display(qc.draw(output="mpl", scale=1.5))

display_array(sv1, prefix=r"\mathbf{Statevector\;1}=")
display(plot_bloch_multivector(sv1))  # type: ignore

display_array(sv2, prefix=r"\mathbf{Statevector\;2}=")
display(plot_bloch_multivector(sv2))  # type: ignore

In [ ]:
# Cell 3 - One qubit circuit not on basis (no measurement)

backend: Any = Aer.get_backend("statevector_simulator")

qc: Any = QuantumCircuit(1)

qc.initialize([0.5, np.sqrt(3) / 2], 0)
qc.save_statevector("sv1")

qc.x(0)
qc.save_statevector("sv2")

result: Any = execute(qc, backend).result()
counts: Any = result.get_counts()

sv1: Any = result.data(0)["sv1"]
sv2: Any = result.data(0)["sv2"]

display(qc.draw(output="mpl", scale=1.5))

display_array(sv1, prefix=r"\mathbf{Statevector\;1}=")
display(plot_bloch_multivector(sv1))  # type: ignore

display_array(sv2, prefix=r"\mathbf{Statevector\;2}=")
display(plot_bloch_multivector(sv2))  # type: ignore

display(plot_distribution(counts))  # type: ignore

In [ ]:
# Cell 4 - One Pete Box (no measurement)

backend: Any = Aer.get_backend("statevector_simulator")

qc: Any = QuantumCircuit(1)
qc.initialize([0, 1], 0)
qc.save_statevector("sv1")
qc.h(0)
qc.save_statevector("sv2")

result: Any = execute(qc, backend, shots=1).result()
counts: Any = result.get_counts()

sv1: Any = result.data(0)["sv1"]
sv2: Any = result.data(0)["sv2"]

display(qc.draw(output="mpl", scale=1.5))

display_array(sv1, prefix=r"\mathbf{Statevector\;1}=")
display(plot_bloch_multivector(sv1))  # type: ignore

display_array(sv2, prefix=r"\mathbf{Statevector\;2}=")
display(plot_bloch_multivector(sv2))  # type: ignore

plot_distribution(counts)

In [ ]:
# Cell 5 - Two Pete Boxes (no measurement)
# Note: This demonstrates the Hadamard Gate is Unitary

backend: Any = Aer.get_backend("statevector_simulator")

qc: Any = QuantumCircuit(1)
qc.initialize([0, 1], 0)
qc.save_statevector("sv1")
qc.h(0)
qc.save_statevector("sv2")
qc.h(0)
qc.save_statevector("sv3")

result: Any = execute(qc, backend).result()
counts: Any = result.get_counts()

sv1: Any = result.data(0)["sv1"]
sv2: Any = result.data(0)["sv2"]
sv3: Any = result.data(0)["sv3"]

display(qc.draw(output="mpl", scale=1.5))

display_array(sv1, prefix=r"\mathbf{Statevector\;1}=")
display(plot_bloch_multivector(sv1))  # type: ignore

display_array(sv2, prefix=r"\mathbf{Statevector\;2}=")
display(plot_bloch_multivector(sv2))  # type: ignore

display_array(sv3, prefix=r"\mathbf{Statevector\;3}=")
display(plot_bloch_multivector(sv3))  # type: ignore

display(plot_distribution(counts))  # type: ignore

In [ ]:
# Cell 6 - Gate Equivalence (no measurement)
# This demonstrates the identity X = HZH

backend: Any = Aer.get_backend("statevector_simulator")

qc: Any = QuantumCircuit(1)

qc.initialize([0, 1], 0)
qc.save_statevector("sv1")
qc.h(0)
qc.save_statevector("sv2")
qc.z(0)
qc.save_statevector("sv3")
qc.h(0)
qc.save_statevector("sv4")

result: Any = execute(qc, backend).result()
counts: Any = result.get_counts()

sv1: Any = result.data(0)["sv1"]
sv2: Any = result.data(0)["sv2"]
sv3: Any = result.data(0)["sv3"]
sv4: Any = result.data(0)["sv4"]

display(qc.draw(output="mpl", scale=1.5))

# display(array_to_latex(sv1, prefix="\\text{Statevector 1} = "))
display_array(sv1, prefix=r"\mathbf{Statevector\;1}=")
display(plot_bloch_multivector(sv1))  # type: ignore

display_array(sv2, prefix=r"\mathbf{Statevector\;2}=")
display(plot_bloch_multivector(sv2))  # type: ignore

display_array(sv3, prefix=r"\mathbf{Statevector\;3}=")
display(plot_bloch_multivector(sv3))  # type: ignore

display_array(sv4, prefix=r"\mathbf{Statevector\;4}=")
display(plot_bloch_multivector(sv4))  # type: ignore

In [ ]:
# Cell 7 - One qubit circuit not on basis (10 shots)

backend: Any = Aer.get_backend("aer_simulator")

qc: Any = QuantumCircuit(1)
qc.initialize([0.5, np.sqrt(3) / 2], 0)
qc.x(0)

qc.measure_all()

result: Any = execute(qc, backend, shots=10).result()
counts: Any = result.get_counts()

display(qc.draw(output="mpl", scale=1.5))

display(plot_distribution(counts))  # type: ignore

In [ ]:
# Cell 8 - One qubit circuit not on basis (10,000 shots)

backend: Any = Aer.get_backend("aer_simulator")

qc: Any = QuantumCircuit(1)
qc.initialize([0.5, np.sqrt(3) / 2], 0)
qc.x(0)

qc.measure_all()

result: Any = execute(qc, backend, shots=10_000).result()
counts: Any = result.get_counts()

display(plot_distribution(counts))  # type: ignore